<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/cifar_10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

def create_model(input_image):
  """
  Defines the neural network architecture for CIFAR-10 classification.
  
  Parameters:
    input_image - The image to classify. Shape: (32 x 32 x 3)
  """
  input_shape = (1, *input_image.shape)
  (N, H, W, C) = input_shape
  
  with tf.variable_scope("conv1") as scope:
    img = tf.placeholder(tf.float64, shape=input_shape)
    w = tf.get_variable("W", [H,W,C,16], initializer=tf.random_normal_initializer)
    b = tf.get_varialbe("b", [16], initializer=tf.zeros_initializer)
    strides = [1, 1, 1, 1]
    padding = "SAME"
    
    conv = tf.nn.conv2d(img, w, strides, padding)
    conv1 = tf.nn.relu(conv + b, name=scope.name)
  
  with tf.variable_scope("conv2") as scope:
    inputs = tf.placeholder(tf.float64, shape=tf.shape(conv1))
    w = tf.get_variable("W", [H,W,16,32], initializer=tf.random_normal_initializer)
    b = tf.get_varialbe("b", [32], initializer=tf.zeros_initializer)
    strides = [1, 1, 1, 1]
    padding = "VALID"
    
    conv = tf.nn.conv2d(inputs, w, strides, padding)
    conv2 = tf.nn.relu(conv + b, name=scope.name)
    
  with tf.Session() as sess:
    output = sess.run(conv2)

In [0]:
import numpy as np

input_image = np.arange(36).reshape(3, 4, 3)
create_model(input_image)

Input shape: (1, 3, 4, 3)
